In [ ]:
import numpy as np
import pandas as pd

import os
import pdfplumber
import secrets
import sys

from dotenv import load_dotenv, find_dotenv

# environment settings
load_dotenv(find_dotenv())
ROOT_DIR = os.path.dirname(find_dotenv())
sys.path.append(ROOT_DIR)

# pandas display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [ ]:
# pdf variables
pdf_name = 'daily-long'
pdf_path = f'{ROOT_DIR}/data/{pdf_name}.pdf'

# csv variables
csv_path = f'{ROOT_DIR}/data/{pdf_name}.csv'
csv_headers = ['symbol', 'date', 'side', 'quantity', 'price', 'commission', 'fees']

In [ ]:
pdf = pdfplumber.open(pdf_path)

# first page
page = pdf.pages[0]
tb = page.extract_table()

# second page
page2 = pdf.pages[1]
tb2 = page2.extract_table()

In [ ]:
for page in pdf.pages:
    print(page.extract_table())

In [ ]:
# first page of table
# df = pd.DataFrame(tb[1:], columns=tb[0])
# df.head(10)

# second page of table loaded with headers from first page of table
df = pd.DataFrame(tb2, columns = tb[0])
df.head(10)

In [ ]:
# clean up initial pdf extraction
df = df.rename(columns = {df.columns[-4]:'value'})
df.loc[df['Comm/Fee/Tax'] == '0.00', 'value'] = 0.00

# concatenate date and time 
df.loc[:, 'date'] = df['Trade Date'] + ' ' + df['Time'] + '00'

# select columns to keep and rename
df = df[['Symbol & Name', 'date', 'Buy/Sell', 'Quantity', 'Traded Price', 'Comm/Fee/Tax', 'value']]
df = df.rename(columns = {'Symbol & Name': 'symbol', 
                          'Buy/Sell': 'side', 
                          'Quantity': 'quantity', 
                          'Traded Price': 'price'})

df.head()

In [ ]:
# extract symbol
df.loc[:, 'symbol'] = df['symbol'].str.extract('(^.+(?=\\n))')
df.head()

In [ ]:
# convert date to UTC with Zulu format
df['date'] = df['date'].str.replace('GMT', '')
df.loc[:, 'date'] = pd.to_datetime(df['date'], format = '%d/%m/%Y %H:%M:%S,%z').dt.tz_convert('UTC')
df.loc[:, 'date'] = df.loc[:, 'date'].apply(lambda x: x.isoformat().replace('+00:00', 'Z'))
df['date'] = np.where(df['date'] == 'NaT', None, df['date'])

df.head()

In [ ]:
# set unique ids before pivot
df['unique_id'] = df['symbol'].apply(lambda x: secrets.token_hex(8) if pd.notna(x) else None)
df = df[['unique_id', 'symbol', 'date', 'side', 'quantity', 'price', 'Comm/Fee/Tax', 'value']]
df.head()

In [ ]:
# forward fill in preparation for pivot
df.iloc[:, :-2] = df.iloc[:, :-2].ffill()
df

In [ ]:
# pivot table
df_p = df.pivot(index = df.columns[:-2].to_list(), columns = 'Comm/Fee/Tax', values = 'value').reset_index()
df_p.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_p.columns]

# replace NaNs with 0 for SEC and TAF
df_p['SEC'] = df_p['SEC'].fillna(0)
df_p['TAF'] = df_p['TAF'].fillna(0)

# select columns to keep and rename
df_p = df_p[['symbol', 'date', 'side', 'quantity', 'price', 'SEC', 'TAF']]
df_p = df_p.rename(columns = {'SEC': 'commission', 'TAF': 'fees'})

# sort by date
df_p.sort_values(by = ['date'], inplace = True)
df_p.reset_index(drop = True, inplace = True)

df_p

In [ ]:
# save csv file
if not os.path.exists(csv_path):
    df_p.to_csv(csv_path)

df_p.head()